In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from prophet import Prophet
import holidays as vacay
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
from pandas.plotting import autocorrelation_plot
from prophet.utilities import regressor_coefficients

In [55]:
output_intermediate = '../output_intermediate'
file = '../data/new_5sta(DNOWS)_3com(NO2,PM10K,O3)_year2010_TemporalDummies_Ox_holidays_meteo.csv'
df_org = pd.read_csv(file, index_col=0, parse_dates=True)
# df_org.columns

In [3]:
features=['Temp', 'RH', 'Pressure', 
          'Winddirection', 'Windspeed', 'Precip']

features2=['weekday_Friday', 'weekday_Monday',
       'weekday_Saturday', 'weekday_Sunday', 'weekday_Thursday',
       'weekday_Tuesday', 'weekday_Wednesday']

features3 = ['season_fall', 'season_spring',
             'season_summer', 'season_winter'
]

In [4]:
years = df_org['year'].unique()
years

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021])

In [5]:
AREAS = ['D_', 'N_', 'O_', 'S_', 'W_']
factors = ['NO2', 'PM10K', 'O3', 'Ox']
add_names = {
    'D_': 'DonBosco_',
    'N_': 'Nord_',
    'O_': 'Ost_',
    'W_': 'West_',
    'S_': 'Sud_'
}

# lockdowns
s1 = '2020-03-10'
h1 = '2020-03-16'
e1 = '2020-05-02'

s2 = '2020-11-03'
h2 = '2020-11-17'
e2 = '2020-12-06'

s3 = '2020-12-26'
h3 = '2020-12-26'
e3 = '2021-02-07'

In [96]:
tmp = pd.DataFrame()
for v in add_names.values():
    cols = pd.DataFrame([c for c in df_org.columns if v in c], columns=[v[0]])
    tmp = tmp.join(cols, how='outer')
tmp

,D,N,O,W,S
0,DonBosco_RH,Nord_Precip,Ost_Pressure,West_RH,Sud_RH
1,DonBosco_Temp,Nord_Pressure,Ost_RH,West_Temp,Sud_Temp
2,NaN,Nord_RH,Ost_Temp,West_Winddirection,Sud_Winddirection
3,NaN,Nord_Temp,Ost_Winddirection,West_Windspeed,Sud_Windspeed
4,NaN,Nord_Winddirection,Ost_Windspeed,NaN,NaN
5,NaN,Nord_Windspeed,NaN,NaN,NaN


# Cutoffs

In [10]:
cols = [k for k in [i+j for j in factors for i in add_names.keys()]
        if k in df_org.columns
       ]
print(cols)
targets = df_org[cols]
quantiles = targets.quantile(0.98)
for c in cols:
    df_org[c] = [i if i < quantiles[c] else quantiles[c] 
                 for i in df_org[c]]

['D_NO2', 'N_NO2', 'O_NO2', 'W_NO2', 'S_NO2', 'D_PM10K', 'N_PM10K', 'O_PM10K', 'W_PM10K', 'S_PM10K', 'N_O3', 'S_O3', 'N_Ox', 'S_Ox']


In [120]:
def prophet_prediction(df_org, F, periods=None, holidays=None, plot=True, 
                       features=None, features2=None, features3=False,
                       seasonality_mode='additive',
                       s1=None, e1=None, tunning=False
                      ):
    year_NO2 = pd.DataFrame()
    coeff = {}
    if periods is None:
        periods = (datetime.datetime.strptime(e1, '%Y-%m-%d') - 
                   datetime.datetime.strptime(s1, '%Y-%m-%d')).days + 1        
    
    for area in AREAS:
        print(area)
        if area == 'O_':
            a = df_org.iloc[:, -5:].dropna(how='all').index
            df_org = df_org.loc[a].copy()
            print(df_org.shape)
        else:
            df_org = df_org.copy()
        if features is not None:
            try:
                add_features = [add_names[area] + feature for feature in features
                                if add_names[area] + feature in df_org.columns
                               ]
            except:
                pass
        # find appropriate features3 list
        if features3:
            features3 = [f+'_2' for f in add_features]
            
        col = area + F
        if col not in df_org: continue   
        
        if holidays is not None:
            m = Prophet(holidays=holidays, 
                        seasonality_mode=seasonality_mode)        
        else:
            m = Prophet(
#                changepoint_prior_scale=0.01,
#                holidays_prior_scale=0.25,
#                mcmc_samples=300,
#                yearly_seasonality=10,
#                daily_seasonality=False,
               seasonality_mode=seasonality_mode
            )
        
        if s1 and e1:
            df = df_org[:e1][[col]].reset_index()     
        else:
            df = df_org[[col]].reset_index()            
        df.columns = ['ds', 'y']
        
        # add features to data and to additional regressor
        if features is not None:
            for add_feature in add_features:
                if s1 and e1:
                    df[add_feature] = df_org[:e1][add_feature].values
                else:
                    df[add_feature] = df_org[add_feature].values
                m.add_regressor(add_feature, #prior_scale=0.5, 
                                standardize=True)
        if features2 is not None:
            for f in features2:
                if s1 and e1:
                    df[f] = df_org[:e1][f].values
                else:
                    df[f] = df_org[f].values
                m.add_regressor(f, #prior_scale=0.5, 
                                standardize=True)
        if features3 and len(features3)>0:
            for f in features3:
                if s1 and e1:
                    df[f] = df_org[:e1][f].values
                else:
                    df[f] = df_org[f].values
                m.add_regressor(f, #prior_scale=0.5, 
                                standardize=True)
                
        if s1 and e1:
            train = df.set_index('ds', drop=True)[:s1].iloc[:-1, :].reset_index()
        else:
            train = df.iloc[:-periods, :]
        
        # remove NAs
        check_NA = train.isna().sum()
        if check_NA[check_NA>0].shape[0]>0:
            print(f'check NA\n{check_NA}')
            train = train.dropna(how='any').reset_index(drop=True)     

        df = pd.concat([train, df.set_index('ds', drop=True).loc[s1:].reset_index()],
                       ignore_index=True
                      )    
        
        m.fit(train)
        
        future = m.make_future_dataframe(periods=periods)
        future = future[future['ds'].isin(df['ds'])]

        if features is not None:
            coeff[col] = regressor_coefficients(m)
            for add_feature in add_features:
                future[add_feature] = df[add_feature].values
        if features2 is not None:
            for f in features2:
                future[f] = df[f].values
        if features3 and len(features3)>0:
            for f in features3:
                future[f] = df[f].values
                
        for c in future.columns:
            if future[c].isna().sum()!=0:
                future[c] = future[c].fillna(future[c].mean())
        forecast = m.predict(future)
        if plot:
            fig1 = m.plot(forecast)
            plt.title(col)
            plt.show()
                      
        forecast = forecast.set_index('ds').join(df[['ds', 'y']].set_index('ds')).iloc[-periods:, :]
#         forecast = pd.concat([forecast.loc[:'2020-03-25'], forecast.loc['2020-03-31':]])

        y_true = forecast['y']
        y_pred = forecast['yhat']
#         forecast.to_csv('forecast.csv')
        year_NO2[col] = [mean_squared_error(y_true, y_pred, squared=False), 
                      mean_absolute_error(y_true, y_pred)]
        if plot:
            plt.figure(figsize=(10, 3))
            plt.plot(y_true)
            plt.plot(y_pred, 'r')
            plt.title(f'year - {col} - Prophet Modified')
            plt.xticks(rotation=90)
            plt.legend(['true', 'predict'])
            plt.show()
    year_NO2.index = ['RMSE', 'MAE']
    if features is not None:
        return year_NO2, coeff
    return year_NO2

In [2]:
help(Prophet)

Help on class Prophet in module prophet.forecaster:

class Prophet(builtins.object)
 |  Prophet(growth='linear', changepoints=None, n_changepoints=25, changepoint_range=0.8, yearly_seasonality='auto', weekly_seasonality='auto', daily_seasonality='auto', holidays=None, seasonality_mode='additive', seasonality_prior_scale=10.0, holidays_prior_scale=10.0, changepoint_prior_scale=0.05, mcmc_samples=0, interval_width=0.8, uncertainty_samples=1000, stan_backend=None)
 |  
 |  Prophet forecaster.
 |  
 |  Parameters
 |  ----------
 |  growth: String 'linear' or 'logistic' to specify a linear or logistic
 |      trend.
 |  changepoints: List of dates at which to include potential changepoints. If
 |      not specified, potential changepoints are selected automatically.
 |  n_changepoints: Number of potential changepoints to include. Not used
 |      if input `changepoints` is supplied. If `changepoints` is not supplied,
 |      then n_changepoints potential changepoints are selected uniformly 

In [25]:
df = pd.DataFrame()

# 1. baseline additive - f0_add

In [ ]:
method = 'f0_add'
base_add = pd.DataFrame()
for F in factors:
    r = prophet_prediction(df_org=df_org, F=F, 
                           s1=s1, e1=e1, 
                           plot=False
                      )    
    base_add = pd.concat([base_add, r], axis=1)    
base_add.index = base_add.index + '_' + method
df = pd.concat([df, base_add])
df

# 2. baseline multiplicative - f0_mul

In [ ]:
method = 'f0_mul'
base_mul = pd.DataFrame()
for F in factors:
    r = prophet_prediction(df_org=df_org, F=F, 
                           s1=s1, e1=e1,
                           seasonality_mode='multiplicative',
                           plot=False
                      )
    
    base_mul = pd.concat([base_mul, r], axis=1)
base_mul.index = base_mul.index + '_' + method
df = pd.concat([df, base_mul])
df

# 3. add features - f1

In [30]:
coefs = {}

In [ ]:
method = 'f1'
f1 = pd.DataFrame()
for F in factors:
    r, coef = prophet_prediction(df_org=df_org, F=F, 
                           s1=s1, e1=e1, 
                           features=features,
                           seasonality_mode='multiplicative',
                           plot=False
                  )    
    f1 = pd.concat([f1, r], axis=1)
    coefs[f'{method}_{F}'] = coef
f1.index = f1.index + '_' + method
df = pd.concat([df, f1])
df

# 4. add features + features2 (weekdays) - f2

In [ ]:
method = 'f2'
f2 = pd.DataFrame()
for F in factors:
    r, coef = prophet_prediction(df_org=df_org, F=F, 
                           s1=s1, e1=e1, 
                           features=features,
                           features2=features2,
                           seasonality_mode='multiplicative',
                           plot=False
                  )
    
    f2 = pd.concat([f2, r], axis=1)
    coefs[f'{method}_{F}'] = coef
f2.index = f2.index + '_' + method
df = pd.concat([df, f2])
df

# 5. add holidays + multi + features

In [ ]:
holiday_normal = pd.DataFrame(df_org[df_org['holiday']==1].index, columns=['ds'])
holiday_normal['holiday'] = 'holiday'
holiday_school = pd.DataFrame(df_org[df_org['holiday_school']==1].index, columns=['ds'])
holiday_school['holiday'] = 'holiday_school'
a = pd.concat([holiday_normal, holiday_school]).sort_values('ds')
a = a.groupby('ds')['holiday'].agg([len, list])
a['holiday'] = [i[0] if len(i)==1 else 'holiday' for i in a['list']]
holidays = a[['holiday']].reset_index()

holidays['lower_window'] = 0
holidays['upper_window'] = 1

holidays

In [ ]:
method = 'f3'
f3 = pd.DataFrame()
for F in factors:
    r, coef = prophet_prediction(df_org=df_org, F=F, s1=s1, e1=e1, 
                       features=features,
                         holidays=holidays,
                       seasonality_mode='multiplicative', 
                                 plot=False
                      )
    f3 = pd.concat([f3, r], axis=1)
    coefs[f'{method}_{F}'] = coef
f3.index = f3.index + '_' + method
df = pd.concat([df, f3])
df

# 6. add features + feature2 + features3 (seasons) - f4

In [45]:
method = 'f4'
f4 = pd.DataFrame()
for F in factors:
    r, coef = prophet_prediction(df_org=df_org, F=F, s1=s1, e1=e1, 
                       features=features,
                     features2=features2+features3,
                         holidays=holidays,
                       seasonality_mode='multiplicative', 
                                 plot=False
                      )
    f4 = pd.concat([f4, r], axis=1)
    coefs[f'{method}_{F}'] = coef
f4.index = f4.index + '_' + method
df = pd.concat([df, f4])
df

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


D_


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


N_


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


W_
D_
N_


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


O_
(4206, 63)
S_
W_


,D_NO2,N_NO2,O_NO2,S_NO2,W_NO2,D_PM10K,N_PM10K,O_PM10K,S_PM10K,W_PM10K,N_O3,S_O3,N_Ox,S_Ox
RMSE_f0_add,10.143455,5.973273,7.337199,7.411119,7.591715,7.442381,6.228861,6.629419,7.348041,6.960648,12.724419,13.087098,11.461235,10.272690
MAE_f0_add,7.580813,4.369344,5.575182,5.313645,5.846201,6.314402,5.102810,5.037827,5.881646,5.820787,10.666480,10.731256,9.524523,8.758226
RMSE_f0_mul,10.518234,6.209995,8.632411,7.770042,7.582705,7.329732,6.120502,6.884109,7.412393,6.891677,12.567588,13.000201,11.447974,11.091998
MAE_f0_mul,8.094826,4.530561,7.091719,5.690664,5.790493,6.039981,4.947906,5.278543,5.896360,5.773746,10.525150,10.660706,9.528293,9.502931
RMSE_f1,9.518675,6.432249,8.757906,6.432151,6.943755,7.695378,5.964945,7.119738,6.409920,6.525401,9.328356,9.696512,8.090767,8.434828
MAE_f1,7.204071,4.906298,7.888533,4.970178,5.228646,6.337155,5.057705,5.685924,5.171174,5.502574,7.235389,7.810971,6.677317,6.753732
RMSE_f2,9.592260,6.433737,8.785922,6.440651,6.929287,7.705172,5.952094,7.088593,6.351230,6.530490,9.306200,9.678042,8.124657,8.488120
MAE_f2,7.291561,4.907365,7.916838,4.977158,5.217883,6.345273,5.043664,5.652254,5.117400,5.510025,7.223101,7.799322,6.697577,6.802985
RMSE_f3,9.054068,6.387331,8.766038,6.185890,7.102512,7.471890,5.899130,6.032096,5.651301,6.128519,9.267391,9.115663,8.053288,7.710572
MAE_f3,7.064377,4.836456,7.819009,4.827288,5.458463,6.143310,4.970440,4.789593,4.405530,4.946865,7.307246,7.446295,6.661309,5.890993


# 7. feature cross

In [ ]:
weekday_convert = {
    'weekday_Sunday': 0,    
    'weekday_Monday': 1,
    'weekday_Tuesday': 2, 
    'weekday_Wednesday': 3,
    'weekday_Thursday': 4,
    'weekday_Friday': 5,
    'weekday_Saturday': 6
}
for c in features2:
    df_org[c] = [weekday_convert[c] if i else i for i in df_org[c]]
df_org['dayoftheweek'] = df_org[features2].sum(axis=1)
df_org[['dayoftheweek']]

In [ ]:
season_convert = {
    'season_spring': 1,
    'season_summer': 2,
    'season_fall': 3,
    'season_winter': 4
}

for c in season_convert.keys():
    df_org[c] = [season_convert[c] if i else i for i in df_org[c]]
df_org['season'] = df_org[season_convert.keys()].sum(axis=1)
df_org[['season']]

In [ ]:
df_org['cross1'] = df_org['season']*df_org['dayoftheweek']
df_org['cross2'] = df_org['season']**2 + df_org['dayoftheweek']**2
df_org['cross3'] = df_org['season']**2
df_org['cross4'] = df_org['dayoftheweek']**2
crosses = ['cross1', 'cross2', 'cross3', 'cross4']
df_org[crosses]

In [50]:
method = 'f5'
f5 = pd.DataFrame()
for F in factors:
    r, coef = prophet_prediction(df_org=df_org, F=F, s1=s1, e1=e1, 
            features=features,
            features2=['dayoftheweek', 'season', 'dayofyear']+crosses,
            holidays=holidays,
            seasonality_mode='multiplicative', plot=False
             )
    f5 = pd.concat([f5, r], axis=1)
    coefs[f'{method}_{F}'] = coef
f5.index = f5.index + '_' + method
df = pd.concat([df, f5])
df

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


D_


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


N_


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


W_
D_
N_


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


O_
(4206, 68)
S_
W_


,D_NO2,N_NO2,O_NO2,S_NO2,W_NO2,D_PM10K,N_PM10K,O_PM10K,S_PM10K,W_PM10K,N_O3,S_O3,N_Ox,S_Ox
RMSE_f0_add,10.143455,5.973273,7.337199,7.411119,7.591715,7.442381,6.228861,6.629419,7.348041,6.960648,12.724419,13.087098,11.461235,10.272690
MAE_f0_add,7.580813,4.369344,5.575182,5.313645,5.846201,6.314402,5.102810,5.037827,5.881646,5.820787,10.666480,10.731256,9.524523,8.758226
RMSE_f0_mul,10.518234,6.209995,8.632411,7.770042,7.582705,7.329732,6.120502,6.884109,7.412393,6.891677,12.567588,13.000201,11.447974,11.091998
MAE_f0_mul,8.094826,4.530561,7.091719,5.690664,5.790493,6.039981,4.947906,5.278543,5.896360,5.773746,10.525150,10.660706,9.528293,9.502931
RMSE_f1,9.518675,6.432249,8.757906,6.432151,6.943755,7.695378,5.964945,7.119738,6.409920,6.525401,9.328356,9.696512,8.090767,8.434828
MAE_f1,7.204071,4.906298,7.888533,4.970178,5.228646,6.337155,5.057705,5.685924,5.171174,5.502574,7.235389,7.810971,6.677317,6.753732
RMSE_f2,9.592260,6.433737,8.785922,6.440651,6.929287,7.705172,5.952094,7.088593,6.351230,6.530490,9.306200,9.678042,8.124657,8.488120
MAE_f2,7.291561,4.907365,7.916838,4.977158,5.217883,6.345273,5.043664,5.652254,5.117400,5.510025,7.223101,7.799322,6.697577,6.802985
RMSE_f3,9.054068,6.387331,8.766038,6.185890,7.102512,7.471890,5.899130,6.032096,5.651301,6.128519,9.267391,9.115663,8.053288,7.710572
MAE_f3,7.064377,4.836456,7.819009,4.827288,5.458463,6.143310,4.970440,4.789593,4.405530,4.946865,7.307246,7.446295,6.661309,5.890993


# 8. cross more features?

In [100]:
df_org.columns

Index(['D_NO2', 'D_PM10K', 'N_NO2', 'N_O3', 'N_PM10K', 'O_NO2', 'O_PM10K',
       'S_NO2', 'S_O3', 'S_PM10K', 'W_NO2', 'W_PM10K', 'year', 'dayofyear',
       'month_Apr', 'month_Aug', 'month_Dec', 'month_Feb', 'month_Jan',
       'month_Jul', 'month_Jun', 'month_Mar', 'month_May', 'month_Nov',
       'month_Oct', 'month_Sep', 'weekday_Friday', 'weekday_Monday',
       'weekday_Saturday', 'weekday_Sunday', 'weekday_Thursday',
       'weekday_Tuesday', 'weekday_Wednesday', 'season_fall', 'season_spring',
       'season_summer', 'season_winter', 'N_Ox', 'S_Ox', 'holiday',
       'holiday_school', 'DonBosco_RH', 'DonBosco_Temp', 'Nord_Precip',
       'Nord_Pressure', 'Nord_RH', 'Nord_Temp', 'Nord_Winddirection',
       'Nord_Windspeed', 'Sud_RH', 'Sud_Temp', 'Sud_Winddirection',
       'Sud_Windspeed', 'West_RH', 'West_Temp', 'West_Winddirection',
       'West_Windspeed', 'Ost_Pressure', 'Ost_RH', 'Ost_Temp',
       'Ost_Winddirection', 'Ost_Windspeed', 'dayoftheweek', 'season',
       'cr

In [107]:
cols = [c for c in [j+i for j in add_names.values() for i in features] if c in df_org.columns]
for c in cols:
    df_org[c+'_2'] = df_org[c]**2
df_org[[c+'_2' for c in cols]]    

,DonBosco_Temp_2,DonBosco_RH_2,Nord_Temp_2,Nord_RH_2,Nord_Pressure_2,Nord_Winddirection_2,Nord_Windspeed_2,Nord_Precip_2,Ost_Temp_2,Ost_RH_2,...,Ost_Winddirection_2,Ost_Windspeed_2,West_Temp_2,West_RH_2,West_Winddirection_2,West_Windspeed_2,Sud_Temp_2,Sud_RH_2,Sud_Winddirection_2,Sud_Windspeed_2
2010-01-01,11.432988,6977.608389,16.288736,5967.853582,909722.422160,7997.614007,0.999400,0.0,NaN,NaN,...,NaN,NaN,12.232296,6894.154268,36081.025294,0.106551,6.048646,7419.312301,31994.845373,0.128470
2010-01-02,8.731708,7571.792958,12.406817,6652.711544,902737.629640,12132.370421,0.626956,0.0,NaN,NaN,...,NaN,NaN,9.203047,7530.265084,45940.542473,0.042700,5.445354,7937.696289,38641.612680,0.078254
2010-01-03,7.148960,3089.459328,9.666530,3224.701699,924702.562004,51796.161191,14.013760,0.0,NaN,NaN,...,NaN,NaN,8.724564,3190.980684,77695.597364,6.971814,5.935762,3299.425286,69861.435977,3.585978
2010-01-04,0.446713,1770.078091,0.551762,2411.853586,948984.198627,58806.133600,1.638912,0.0,NaN,NaN,...,NaN,NaN,0.326851,2008.048395,81687.161749,1.047181,1.084830,1900.669635,82949.587294,1.291973
2010-01-05,15.569268,5325.319976,14.956557,4806.795187,946250.733618,18042.910111,0.496253,0.0,NaN,NaN,...,NaN,NaN,16.134632,5455.269317,39445.515020,0.127354,18.823340,5175.992950,43118.348074,0.183346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-03,344.111812,4424.799977,324.734095,4552.782510,954860.036296,28060.176337,0.226884,0.0,348.652584,5359.185786,...,33339.984538,0.189341,339.075028,3361.111988,70076.890176,0.389826,339.227402,4325.227261,37304.670405,0.454037
2021-07-04,478.730374,2989.452729,453.265590,3399.000771,953242.724622,22682.067836,0.395655,0.0,492.833873,3621.288179,...,17095.099648,0.316920,483.473859,2453.776658,32131.028552,0.332639,474.224358,3006.806379,20001.488845,0.811988
2021-07-05,423.046984,4576.120668,394.684299,4643.476626,948115.253686,11606.726800,0.417967,0.0,429.610477,5129.619208,...,19528.595153,0.174956,425.355151,3380.213786,30337.414833,0.297901,426.623696,4033.790657,28382.093728,0.402665
2021-07-06,471.984272,4309.877858,466.030518,4360.409916,947878.072254,14486.296103,0.726703,0.0,477.715772,5567.192290,...,16757.680496,0.213591,477.809280,3319.924500,39219.029640,0.458950,473.272933,4186.575264,26397.826672,1.299849


In [122]:
method = 'f6'
f6 = pd.DataFrame()
for F in factors:
    r, coef = prophet_prediction(df_org=df_org, F=F, s1=s1, e1=e1, 
                                features=features,
                                features2=['dayoftheweek', 'season', 'dayofyear']+crosses,
                                features3=True,
                                holidays=holidays,
                                seasonality_mode='multiplicative', plot=False
                                 )
    f6 = pd.concat([f6, r], axis=1)
    coefs[f'{method}_{F}'] = coef
f6.index = f6.index + '_' + method
df = pd.concat([df, f6])
df

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


D_


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


N_


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


W_
D_
N_


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


O_
(4206, 89)
S_
W_


,D_NO2,N_NO2,O_NO2,S_NO2,W_NO2,D_PM10K,N_PM10K,O_PM10K,S_PM10K,W_PM10K,N_O3,S_O3,N_Ox,S_Ox
RMSE_f0_add,10.143455,5.973273,7.337199,7.411119,7.591715,7.442381,6.228861,6.629419,7.348041,6.960648,12.724419,13.087098,11.461235,10.272690
MAE_f0_add,7.580813,4.369344,5.575182,5.313645,5.846201,6.314402,5.102810,5.037827,5.881646,5.820787,10.666480,10.731256,9.524523,8.758226
RMSE_f0_mul,10.518234,6.209995,8.632411,7.770042,7.582705,7.329732,6.120502,6.884109,7.412393,6.891677,12.567588,13.000201,11.447974,11.091998
MAE_f0_mul,8.094826,4.530561,7.091719,5.690664,5.790493,6.039981,4.947906,5.278543,5.896360,5.773746,10.525150,10.660706,9.528293,9.502931
RMSE_f1,9.518675,6.432249,8.757906,6.432151,6.943755,7.695378,5.964945,7.119738,6.409920,6.525401,9.328356,9.696512,8.090767,8.434828
MAE_f1,7.204071,4.906298,7.888533,4.970178,5.228646,6.337155,5.057705,5.685924,5.171174,5.502574,7.235389,7.810971,6.677317,6.753732
RMSE_f2,9.592260,6.433737,8.785922,6.440651,6.929287,7.705172,5.952094,7.088593,6.351230,6.530490,9.306200,9.678042,8.124657,8.488120
MAE_f2,7.291561,4.907365,7.916838,4.977158,5.217883,6.345273,5.043664,5.652254,5.117400,5.510025,7.223101,7.799322,6.697577,6.802985
RMSE_f3,9.054068,6.387331,8.766038,6.185890,7.102512,7.471890,5.899130,6.032096,5.651301,6.128519,9.267391,9.115663,8.053288,7.710572
MAE_f3,7.064377,4.836456,7.819009,4.827288,5.458463,6.143310,4.970440,4.789593,4.405530,4.946865,7.307246,7.446295,6.661309,5.890993


## save coefficients

In [135]:
for k, v in coefs.items():
    for k1, v1 in v.items():
        try:
            coefs[k][k1] = v1.to_dict()
        except:
            pass
#         print(type(v1))
#         break
#     break


In [136]:
import json
with open('data.json', 'w') as fp:
    json.dump(coefs, fp,  indent=4)

# A. THE BEST METHOD

In [123]:
RMSE_indices = [i for i in df.index if 'RMSE_' in i]
rmse = df.loc[RMSE_indices]
MAE_indices = [i for i in df.index if 'MAE_' in i]
mae = df.loc[MAE_indices]
df_min = pd.concat([rmse.idxmin(), mae.idxmin()], axis=1)
df_min.columns = ['RMSE', 'MAE']

val_min = pd.concat([rmse.min(), mae.min()], axis=1)
val_min.columns = ['RMSE', 'MAE']

df_min['RMSE_val'] = val_min['RMSE']
df_min['MAE_val'] = val_min['MAE']
df_min = df_min.sort_values('RMSE_val')
df_min['RMSE'] = df_min['RMSE'].apply(lambda x: x.replace('RMSE_', ''))
df_min['MAE'] = df_min['MAE'].apply(lambda x: x.replace('MAE_', ''))
df_min

,RMSE,MAE,RMSE_val,MAE_val
S_PM10K,f5,f5,5.549522,4.242137
O_PM10K,f5,f5,5.758059,4.593578
S_NO2,f6,f6,5.793037,4.606080
N_PM10K,f5,f5,5.835789,4.875501
N_NO2,f0_add,f0_add,5.973273,4.369344
W_PM10K,f5,f5,6.108925,4.909600
W_NO2,f2,f2,6.929287,5.217883
D_PM10K,f0_mul,f0_mul,7.329732,6.039981
O_NO2,f0_add,f0_add,7.337199,5.575182
S_Ox,f6,f5,7.496641,5.755069


In [ ]:
a = df_min[['RMSE_val', 'MAE_val']].stack().reset_index()
a.columns = ['target', 'Error', 'Val']
a = a.sort_values('Val')
a['Val'] = a['Val'].round(2)
px.bar(a, x='target', y='Val', text='Val', color='Error', barmode='group')

# B. COMPARISON

In [ ]:
title = 'RMSE'
factor = 'NO2'
df_mrse = df.loc[RMSE_indices].sort_index().reset_index()
df_mrse['index'] = df_mrse['index'].apply(lambda x: x.replace(f'{title}_', ''))
df_mrse = df_mrse.set_index('index')

cols = [c for c in df_mrse.columns if factor in c]
a = df_mrse[cols]
fig = px.bar(a.T, barmode='group', title=title)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=0.96,
    xanchor="right",
    x=1.02
))
fig.show()

In [126]:
a

,D_NO2,N_NO2,O_NO2,S_NO2,W_NO2
index,,,,,
f0_add,10.143455,5.973273,7.337199,7.411119,7.591715
f0_mul,10.518234,6.209995,8.632411,7.770042,7.582705
f1,9.518675,6.432249,8.757906,6.432151,6.943755
f2,9.592260,6.433737,8.785922,6.440651,6.929287
f3,9.054068,6.387331,8.766038,6.185890,7.102512
f4,9.095883,6.442532,8.790350,6.180596,7.149437
f5,8.929430,6.417594,8.830336,6.184703,7.093647
f6,9.053477,6.308132,8.952369,5.793037,7.055068


In [127]:
pd.concat([a.min(), a.idxmin()], axis=1)

,0,1
D_NO2,8.929430,f5
N_NO2,5.973273,f0_add
O_NO2,7.337199,f0_add
S_NO2,5.793037,f6
W_NO2,6.929287,f2


In [ ]:
title = 'RMSE'
factor = 'PM10K'

cols = [c for c in df_mrse.columns if factor in c]
a = df_mrse[cols]
fig = px.bar(a.T, barmode='group', title=title)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=0.96,
    xanchor="right",
    x=1.02
))
fig.show()

In [129]:
a

,D_PM10K,N_PM10K,O_PM10K,S_PM10K,W_PM10K
index,,,,,
f0_add,7.442381,6.228861,6.629419,7.348041,6.960648
f0_mul,7.329732,6.120502,6.884109,7.412393,6.891677
f1,7.695378,5.964945,7.119738,6.409920,6.525401
f2,7.705172,5.952094,7.088593,6.351230,6.530490
f3,7.471890,5.899130,6.032096,5.651301,6.128519
f4,7.489974,5.897957,5.856647,5.662259,6.114597
f5,7.467168,5.835789,5.758059,5.549522,6.108925
f6,9.105950,6.123103,6.312552,6.605101,6.336099


In [ ]:
title = 'RMSE'
factor = 'O3'

cols = [c for c in df_mrse.columns if factor in c]
a = df_mrse[cols]
fig = px.bar(a.T, barmode='group', title=title)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=0.98,
    xanchor="right",
    x=1.02
))
fig.show()

In [131]:
a

,N_O3,S_O3
index,,
f0_add,12.724419,13.087098
f0_mul,12.567588,13.000201
f1,9.328356,9.696512
f2,9.306200,9.678042
f3,9.267391,9.115663
f4,9.178053,9.025235
f5,9.169308,8.967940
f6,8.884625,8.476634


In [ ]:
title = 'RMSE'
factor = 'Ox'

cols = [c for c in df_mrse.columns if factor in c]
a = df_mrse[cols]
fig = px.bar(a.T, barmode='group', title=title)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=0.97,
    xanchor="right",
    x=1.02
))
fig.show()

In [133]:
a

,N_Ox,S_Ox
index,,
f0_add,11.461235,10.272690
f0_mul,11.447974,11.091998
f1,8.090767,8.434828
f2,8.124657,8.488120
f3,8.053288,7.710572
f4,8.054771,7.655022
f5,8.005494,7.595386
f6,7.958320,7.496641


In [ ]:
df.round(2)

In [134]:
df.to_csv('../output/20210817_results_new_data_cutoffs.csv')